# Imports

In [1]:
import requests
import folium
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import KNNImputer
from wordcloud import WordCloud, STOPWORDS
from statsmodels.tsa.seasonal import seasonal_decompose

# Mounting GDrive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/BDM_CAPSTONE_PROJECT/

/content/drive/MyDrive/BDM_CAPSTONE_PROJECT


# Load Data

In [4]:
fuel = pd.read_csv('DATA/fuel.csv')
data = pd.read_csv('DATA/data.csv')
location = pd.read_csv('DATA/location.csv')
weather = pd.read_csv('DATA/weather.csv')

In [5]:
fuel.shape, data.shape, weather.shape, location.shape

((722, 2), (722, 7), (722, 4), (29, 3))

In [6]:
data.head(2)

,DATE,DEMAND,FROM,TO,DISTANCE,PROFIT,LAGAN
0,2024-07-22,1.0,Masaurhi,NaN,132.0,1000.0,0.0
1,2024-07-21,1.0,Masaurhi,NaN,46.0,1000.0,0.0


In [7]:
fuel.head(2)

,DATE,FUEL_COST_PER_LITRE
0,2024-07-22,92.03
1,2024-07-21,92.03


In [8]:
weather.head(2)

,DATE,TMIN,TMAX,PRCP
0,2022-08-01,25.8,33.6,0.5
1,2022-08-02,26.0,33.0,7.1


In [9]:
location.head(2)

,LOCATION,LATITUDE,LONGITUDE
0,Punpun,25.463909,85.112189
1,Fatehpur,24.608398,85.226437


# Null Count

In [10]:
fuel.isna().sum()

,0
DATE,0
FUEL_COST_PER_LITRE,0


In [11]:
weather.isna().sum()

,0
DATE,0
TMIN,0
TMAX,0
PRCP,0


In [12]:
data[data.DEMAND==1].isna().sum()

,0
DATE,0
DEMAND,0
FROM,0
TO,278
DISTANCE,196
PROFIT,128
LAGAN,0


# Merge Dataframes

In [13]:
merged_df = data.merge(fuel, on='DATE').merge(weather, on='DATE')
merged_df.head(2)

,DATE,DEMAND,FROM,TO,DISTANCE,PROFIT,LAGAN,FUEL_COST_PER_LITRE,TMIN,TMAX,PRCP
0,2024-07-22,1.0,Masaurhi,NaN,132.0,1000.0,0.0,92.03,31.0,36.0,0.6
1,2024-07-21,1.0,Masaurhi,NaN,46.0,1000.0,0.0,92.03,31.0,35.0,0.4


In [14]:
merged_df.isna().sum()

,0
DATE,0
DEMAND,0
FROM,372
TO,650
DISTANCE,568
PROFIT,500
LAGAN,0
FUEL_COST_PER_LITRE,0
TMIN,0
TMAX,0


# Preprocessing

In [15]:
df_filtered = merged_df[merged_df['DEMAND'] == 1].copy()

In [16]:
df_filtered.isna().sum()

,0
DATE,0
DEMAND,0
FROM,0
TO,278
DISTANCE,196
PROFIT,128
LAGAN,0
FUEL_COST_PER_LITRE,0
TMIN,0
TMAX,0


## Imputing Null Values

In [17]:
df_filtered['TO'] = df_filtered['TO'].fillna('Unknown')

In [18]:
df_filtered.isna().sum()

,0
DATE,0
DEMAND,0
FROM,0
TO,0
DISTANCE,196
PROFIT,128
LAGAN,0
FUEL_COST_PER_LITRE,0
TMIN,0
TMAX,0


In [19]:
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(df_filtered[['FROM', 'TO']])
encoded_data = enc.transform(df_filtered[['FROM', 'TO']]).toarray()
encoded_df = pd.DataFrame(encoded_data, columns=enc.get_feature_names_out(['FROM', 'TO']))
df_encoded = pd.concat([df_filtered, encoded_df], axis=1)
df_encoded['DATE'] = pd.to_datetime(df_encoded['DATE'])
df_encoded['YEAR'] = df_encoded['DATE'].dt.year
df_encoded['MONTH'] = df_encoded['DATE'].dt.month
df_encoded['DAY'] = df_encoded['DATE'].dt.day
df_encoded = df_encoded.drop(['DATE', 'FROM', 'TO'], axis=1)
imputer = KNNImputer(n_neighbors=5)
df_imputed = imputer.fit_transform(df_encoded)
df_imputed = pd.DataFrame(df_imputed, columns=df_encoded.columns)

df_imputed.head(2)

,DEMAND,DISTANCE,PROFIT,LAGAN,FUEL_COST_PER_LITRE,TMIN,TMAX,PRCP,FROM_Daulatpur,FROM_Diara Malahi,...,TO_Parawalpur,TO_Patna,TO_Piyar pura,TO_Punpun,TO_Rajgir,TO_Siwan,TO_Unknown,YEAR,MONTH,DAY
0,1.0,132.0,1000.0,0.0,92.03,31.0,36.0,0.6,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2024.0,7.0,22.0
1,1.0,46.0,1000.0,0.0,92.03,31.0,35.0,0.4,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2024.0,7.0,21.0


In [20]:
df_filtered['PROFIT'] = df_imputed['PROFIT']
df_filtered['DISTANCE'] = df_imputed['DISTANCE']

In [21]:
df_filtered.isna().sum()

,0
DATE,0
DEMAND,0
FROM,0
TO,0
DISTANCE,96
PROFIT,96
LAGAN,0
FUEL_COST_PER_LITRE,0
TMIN,0
TMAX,0


In [22]:
df_filtered[pd.isna(df_filtered.DISTANCE)].head()

,DATE,DEMAND,FROM,TO,DISTANCE,PROFIT,LAGAN,FUEL_COST_PER_LITRE,TMIN,TMAX,PRCP
511,2023-02-27,1.0,Masaurhi,Unknown,NaN,NaN,1.0,94.02,16.0,31.2,0.0
512,2023-02-26,1.0,Masaurhi,Unknown,NaN,NaN,1.0,94.02,15.2,31.0,0.0
514,2023-02-24,1.0,Masaurhi,Unknown,NaN,NaN,0.0,94.02,16.0,31.0,0.0
515,2023-02-23,1.0,Masaurhi,Unknown,NaN,NaN,1.0,94.02,16.6,32.0,0.0
516,2023-02-22,1.0,Masaurhi,Unknown,NaN,NaN,0.0,94.02,18.0,33.0,0.0


**TO, DISTANCE, and PROFIT are missing for these 96 rows, and since these features are crucial for analysis, using mean imputation for DISTANCE and PROFIT is a practical approach.**

In [23]:
df_filtered['DISTANCE'] = df_filtered['DISTANCE'].fillna(df_filtered['DISTANCE'].mean())
df_filtered['PROFIT'] = df_filtered['PROFIT'].fillna(df_filtered['PROFIT'].mean())

In [24]:
df_filtered.isna().sum()

,0
DATE,0
DEMAND,0
FROM,0
TO,0
DISTANCE,0
PROFIT,0
LAGAN,0
FUEL_COST_PER_LITRE,0
TMIN,0
TMAX,0


In [25]:
df_filtered.to_csv('preprocessed_data.csv', index=False)